In [1]:
import tensorflow as tf
import numpy as np

In [2]:
sess = tf.InteractiveSession()

In [59]:
logits= np.random.randn(10,4)
probs = tf.nn.softmax(logits)
cdist = tf.distributions.Categorical(logits)
sample_action = cdist.sample()
print("sample_action {}".format(sample_action.eval()))
print("prob shape {}".format(tf.shape(probs).eval()))
print("probs {}".format(probs.eval()))
print("reduce_sum of probs {}".format(tf.reduce_sum(probs, axis=1).eval()))
print("reduce_sum of probs * log(probs) {}".format(tf.reduce_sum(probs * tf.log(probs), axis=1).eval()))

sample_action [1 3 0 0 0 3 2 3 2 1]
prob shape [10  4]
probs [[0.05250511 0.11507613 0.43345686 0.39896191]
 [0.23405947 0.36866694 0.24263212 0.15464147]
 [0.43506061 0.22602385 0.05310279 0.28581275]
 [0.62278721 0.18162029 0.09237892 0.10321358]
 [0.66690505 0.16614643 0.10617012 0.0607784 ]
 [0.47990563 0.17025682 0.03672309 0.31311446]
 [0.16691537 0.32921654 0.38473608 0.11913201]
 [0.12581995 0.27618882 0.43824516 0.15974607]
 [0.14783997 0.02847501 0.46850921 0.35517582]
 [0.01824223 0.64242409 0.03558982 0.30374386]]
reduce_sum of probs [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
reduce_sum of probs * log(probs) [-1.13249331 -1.34005373 -1.21205302 -1.05916245 -0.9767024  -1.13869121
 -1.28555103 -1.27072011 -1.10683038 -0.83796813]


In [96]:
states = np.arange(10*84*84*4).reshape(10,84,84,4)
actions = [np.random.randint(4) for i in range(10)]
actions = np.array(actions)#.reshape(10,4)

batch_size = tf.shape(states)[0]
print("states shape {}".format(states.shape))
print("batch_size {}".format(batch_size.eval()))
print("range {}".format(tf.range(batch_size).eval()))    
print("shape of probs[1] {}".format(tf.shape(probs)[1].eval()))

gather_indices = tf.range(batch_size) * tf.shape(probs)[1] + actions
print("gather_indices {}".format(gather_indices.eval()))
print("reshape of probs {}".format(tf.reshape(probs,[-1]).eval()))
reshape_probs = tf.reshape(probs,[-1]).eval()
# print("gather of probs {}".format(tf.gather(reshape_probs,[1,2]).eval()))
selected_action_probs = tf.gather(tf.reshape(probs, [-1]), gather_indices)
print("selected_action_probs {}".format(selected_action_probs.eval()))

states shape (10, 84, 84, 4)
batch_size 10
range [0 1 2 3 4 5 6 7 8 9]
shape of probs[1] 4
gather_indices [ 1  4  9 13 16 21 24 29 33 38]
reshape of probs [0.05250511 0.11507613 0.43345686 0.39896191 0.23405947 0.36866694
 0.24263212 0.15464147 0.43506061 0.22602385 0.05310279 0.28581275
 0.62278721 0.18162029 0.09237892 0.10321358 0.66690505 0.16614643
 0.10617012 0.0607784  0.47990563 0.17025682 0.03672309 0.31311446
 0.16691537 0.32921654 0.38473608 0.11913201 0.12581995 0.27618882
 0.43824516 0.15974607 0.14783997 0.02847501 0.46850921 0.35517582
 0.01824223 0.64242409 0.03558982 0.30374386]
selected_action_probs [0.11507613 0.23405947 0.22602385 0.18162029 0.66690505 0.17025682
 0.16691537 0.27618882 0.02847501 0.03558982]


In [54]:
input_ = 180.
input_ = tf.to_float(input_) / 255.0
print(input_.eval())

0.7058824


- tf.train.RMSPropOptimizer(learning_rate,decay,momentum,epsilon,...)


In [99]:
frame = np.arange(10).reshape(2,5)
outframe = np.stack([frame] * 4, axis=2)
outframe.shape


(2, 5, 4)

In [3]:
state = np.arange(84*84*4).reshape(84,84,4)
state.shape
next_frame = np.zeros(84*84).reshape(84,84)
expand_next_frame = np.expand_dims(next_frame, 2)
print("expand_next_frame {}".format(expand_next_frame.shape))
state = np.append(state[:,:,1:], np.expand_dims(next_frame, 2), axis=2)
state.shape

expand_next_frame (84, 84, 1)


(84, 84, 4)

In [125]:
next_frame

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [126]:
transposed_state = np.transpose(state,(2,0,1))

In [131]:
transposed_state[2]

array([[3.0000e+00, 7.0000e+00, 1.1000e+01, ..., 3.2700e+02, 3.3100e+02,
        3.3500e+02],
       [3.3900e+02, 3.4300e+02, 3.4700e+02, ..., 6.6300e+02, 6.6700e+02,
        6.7100e+02],
       [6.7500e+02, 6.7900e+02, 6.8300e+02, ..., 9.9900e+02, 1.0030e+03,
        1.0070e+03],
       ...,
       [2.7219e+04, 2.7223e+04, 2.7227e+04, ..., 2.7543e+04, 2.7547e+04,
        2.7551e+04],
       [2.7555e+04, 2.7559e+04, 2.7563e+04, ..., 2.7879e+04, 2.7883e+04,
        2.7887e+04],
       [2.7891e+04, 2.7895e+04, 2.7899e+04, ..., 2.8215e+04, 2.8219e+04,
        2.8223e+04]])

In [133]:
def foo(a,b,c,*args):
    print ("a=%s" % (a,))
    print ("b=%s" % (b,))
    print ("c=%s" % (c,))
    print ("args=%s" % (args,))

argtuple = ("testa","testb","testc","excess","more excess")
foo(*argtuple)

a=testa
b=testb
c=testc
args=('excess', 'more excess')


In [134]:
def foo(a,b,c,**args):
    print ("a=%s" % (a,))
    print ("b=%s" % (b,))
    print ("c=%s" % (c,))
    print ("args=%s" % (args,))

argdict = dict(a="testa", b="testb", c="testc", excessarg="string")
foo(**argdict)

a=testa
b=testb
c=testc
args={'excessarg': 'string'}


In [139]:
def foo(args):
    print(args)
    print(*args)
    for _val in zip(*args):
        print(_val)

l = [[1,2,3,4,5]]
foo(l)

[[1, 2, 3, 4, 5]]
[1, 2, 3, 4, 5]
(1,)
(2,)
(3,)
(4,)
(5,)


In [148]:
sess.run(tf.clip_by_global_norm([2.0,5.0], 5.0))

([1.8569534, 4.6423836], 5.3851647)

In [4]:
# coordinator for threads
coord = tf.train.Coordinator()

In [5]:
import itertools as it
count_with_floats = it.count(start=0.5, step=0.75)
list(next(count_with_floats) for _ in range(5))

[0.5, 1.25, 2.0, 2.75, 3.5]

In [16]:
action_onehot = tf.one_hot([0, 1,  0], 2).eval()

In [18]:
print(action_onehot.shape)
print(action_onehot)

(3, 2)
[[1. 0.]
 [0. 1.]
 [1. 0.]]


In [4]:
1e3

1000.0

### reduce_max

[calculate-max-of-a-tensorflow-tensor](https://www.aiworkbox.com/lessons/calculate-max-of-a-tensorflow-tensor)

In [3]:
random_int_var = tf.get_variable("random_int_var",
                                 initializer=tf.random_uniform([2, 3, 4],
                                                               minval=0,
                                                               maxval=20,
                                                               dtype=tf.int32))

In [5]:
init_var = tf.global_variables_initializer()
sess.run(init_var)
print(sess.run(random_int_var))

[[[ 2 15  9 18]
  [ 5  7  7  0]
  [18  3 17  5]]

 [[15 12  6  1]
  [19 14 16  7]
  [10 19 13 14]]]


In [7]:
print(sess.run(tf.reduce_max(random_int_var,axis= -1)))

[[18  7 18]
 [15 19 19]]


[scatternd](https://www.tensorflow.org/api_docs/python/tf/scatter_nd)

![](https://www.tensorflow.org/images/ScatterNd1.png)

In [8]:
indices = tf.constant([[4], [3], [1], [7]])
updates = tf.constant([9, 10, 11, 12])
shape = tf.constant([8])
scatter = tf.scatter_nd(indices, updates, shape)
with tf.Session() as sess:
  print(sess.run(scatter))


[ 0 11  0 10  9  0  0 12]


### boolean_mask

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
batch_item = tf.get_variable("batch_item",
                                 initializer=tf.random_uniform([10, 25],
                                                               minval=0,
                                                               maxval=10,
                                                               dtype=tf.float32))

In [3]:
sess = tf.InteractiveSession()
init_var = tf.global_variables_initializer()
sess.run(init_var)

print(sess.run(batch_item))

[[8.59433    7.9664564  3.3884418  2.959404   8.9796715  0.17459631
  4.686191   1.9722736  3.3611429  2.9821408  0.0342977  5.9760675
  9.6206665  6.4000106  4.7016106  2.161795   9.35574    1.8032289
  0.269202   4.8897624  8.107533   2.8859198  0.43307543 2.3865807
  5.9412646 ]
 [6.1229267  2.2228634  4.574299   2.409619   0.47965288 7.307167
  3.5909688  5.0035763  5.367388   2.881316   0.9480381  8.158475
  2.2852528  0.3503883  9.57233    9.913822   2.3953867  0.18403888
  6.253934   1.0150552  8.429917   2.467041   3.6928594  0.7664454
  1.3392794 ]
 [3.9279854  1.5616298  9.974927   7.2506847  2.507043   0.9481108
  4.4507146  1.252408   5.290909   9.906596   0.8854866  1.1535144
  1.2830997  7.4291325  7.6091633  7.878511   8.549244   8.237146
  7.512783   7.5756035  6.263311   4.9645004  5.836109   1.3225031
  1.1137915 ]
 [7.518853   3.6708832  9.846251   0.84967613 9.1403475  5.657159
  6.496443   9.457251   2.375549   4.906397   8.19791    9.979845
  7.143793   2.6786983 

In [6]:
index = 0
confidences = tf.expand_dims(batch_item[..., index], axis=-1)
class_id = tf.fill(dims=tf.shape(confidences), value=tf.to_float(index))
box_coordinates = batch_item[...,-4:]
single_class = tf.concat([class_id, confidences, box_coordinates], axis=-1)


threshold_met = single_class[:,1] > 5
single_class = tf.boolean_mask(tensor=single_class,
                               mask=threshold_met)

In [7]:
print(sess.run(class_id),sess.run(tf.shape(class_id)))

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]] [10  1]


In [8]:
print(sess.run(single_class),sess.run(tf.shape(single_class)))

[[0.         8.730918   4.609034   0.23979783 9.534487   5.2389145 ]
 [0.         5.041697   5.850618   2.5402284  6.893468   5.334305  ]
 [0.         5.8045235  7.3003397  8.596873   0.28927803 4.547657  ]
 [0.         5.863278   3.895644   7.443939   7.8275275  5.6249332 ]
 [0.         9.993277   7.395463   1.372019   9.579853   3.0407023 ]
 [0.         9.616355   1.629206   8.20386    8.431232   0.9219861 ]] [6 6]


In [9]:
scores = single_class[...,1]

# `tf.image.non_max_suppression()` needs the box coordinates in the format `(ymin, xmin, ymax, xmax)`.
xmin = tf.expand_dims(single_class[...,-4], axis=-1)
ymin = tf.expand_dims(single_class[...,-3], axis=-1)
xmax = tf.expand_dims(single_class[...,-2], axis=-1)
ymax = tf.expand_dims(single_class[...,-1], axis=-1)
boxes = tf.concat(values=[ymin, xmin, ymax, xmax], axis=-1)

maxima_indices = tf.image.non_max_suppression(boxes=boxes,
                                              scores=scores,
                                              max_output_size=10,
                                              iou_threshold=0.45,
                                              name='non_maximum_suppresion')
maxima = tf.gather(params=single_class,
                   indices=maxima_indices,
                   axis=0)

In [10]:
print(sess.run(maxima),sess.run(maxima_indices))

[[0.         9.993277   7.395463   1.372019   9.579853   3.0407023 ]
 [0.         9.616355   1.629206   8.20386    8.431232   0.9219861 ]
 [0.         8.730918   4.609034   0.23979783 9.534487   5.2389145 ]
 [0.         5.863278   3.895644   7.443939   7.8275275  5.6249332 ]
 [0.         5.8045235  7.3003397  8.596873   0.28927803 4.547657  ]
 [0.         5.041697   5.850618   2.5402284  6.893468   5.334305  ]] [4 5 0 3 2 1]


In [11]:
# Make sure `single_class` is exactly `self.nms_max_output_size` elements long.
padded_single_class = tf.pad(tensor=maxima,
                             paddings=[[0, 10 - tf.shape(maxima)[0]], [0, 0]],
                             mode='CONSTANT',
                             constant_values=0.0)

In [12]:
print(sess.run(padded_single_class))

[[0.         9.993277   7.395463   1.372019   9.579853   3.0407023 ]
 [0.         9.616355   1.629206   8.20386    8.431232   0.9219861 ]
 [0.         8.730918   4.609034   0.23979783 9.534487   5.2389145 ]
 [0.         5.863278   3.895644   7.443939   7.8275275  5.6249332 ]
 [0.         5.8045235  7.3003397  8.596873   0.28927803 4.547657  ]
 [0.         5.041697   5.850618   2.5402284  6.893468   5.334305  ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]]


In [4]:
tf_confidence_thresh = 6.0
iou_threshold = 0.45
tf_nms_max_output_size =10
n_classes = 21
# Create a function that filters the predictions for one single class.
def filter_single_class(index):

    # From a tensor of shape (n_boxes, n_classes + 4 coordinates) extract
    # a tensor of shape (n_boxes, 1 + 4 coordinates) that contains the
    # confidnece values for just one class, determined by `index`.
    confidences = tf.expand_dims(batch_item[..., index], axis=-1)
    class_id = tf.fill(dims=tf.shape(confidences), value=tf.to_float(index))
    box_coordinates = batch_item[...,-4:]

    single_class = tf.concat([class_id, confidences, box_coordinates], axis=-1)

    # Apply confidence thresholding with respect to the class defined by `index`.
    threshold_met = single_class[:,1] > tf_confidence_thresh
    single_class = tf.boolean_mask(tensor=single_class,
                                   mask=threshold_met)

    # If any boxes made the threshold, perform NMS.
    def perform_nms():
        scores = single_class[...,1]

        # `tf.image.non_max_suppression()` needs the box coordinates in the format `(ymin, xmin, ymax, xmax)`.
        xmin = tf.expand_dims(single_class[...,-4], axis=-1)
        ymin = tf.expand_dims(single_class[...,-3], axis=-1)
        xmax = tf.expand_dims(single_class[...,-2], axis=-1)
        ymax = tf.expand_dims(single_class[...,-1], axis=-1)
        boxes = tf.concat(values=[ymin, xmin, ymax, xmax], axis=-1)

        maxima_indices = tf.image.non_max_suppression(boxes=boxes,
                                                      scores=scores,
                                                      max_output_size=tf_nms_max_output_size,
                                                      iou_threshold=iou_threshold,
                                                      name='non_maximum_suppresion')
        maxima = tf.gather(params=single_class,
                           indices=maxima_indices,
                           axis=0)
        return maxima

    def no_confident_predictions():
        return tf.constant(value=0.0, shape=(1,6))

    single_class_nms = tf.cond(tf.equal(tf.size(single_class), 0), no_confident_predictions, perform_nms)

    # Make sure `single_class` is exactly `self.nms_max_output_size` elements long.
    padded_single_class = tf.pad(tensor=single_class_nms,
                                 paddings=[[0, tf_nms_max_output_size - tf.shape(single_class_nms)[0]], [0, 0]],
                                 mode='CONSTANT',
                                 constant_values=0.0)

    return padded_single_class

In [5]:
# Iterate `filter_single_class()` over all class indices.
filtered_single_classes = tf.map_fn(fn=lambda i: filter_single_class(i),
                                    elems=tf.range(1,n_classes),
                                    dtype=tf.float32,
                                    parallel_iterations=128,
                                    back_prop=False,
                                    swap_memory=False,
                                    infer_shape=True,
                                    name='loop_over_classes')

In [7]:
print(sess.run(filtered_single_classes),sess.run(tf.shape(filtered_single_classes)))

[[[ 1.          8.585948    1.2325013   1.1379397   8.475794
    7.276685  ]
  [ 1.          7.9664564   2.8859198   0.43307543  2.3865807
    5.9412646 ]
  [ 0.          0.          0.          0.          0.
    0.        ]
  ...
  [ 0.          0.          0.          0.          0.
    0.        ]
  [ 0.          0.          0.          0.          0.
    0.        ]
  [ 0.          0.          0.          0.          0.
    0.        ]]

 [[ 2.          9.974927    4.9645004   5.836109    1.3225031
    1.1137915 ]
  [ 2.          9.846251    5.084606    2.0820034   7.175162
    6.472485  ]
  [ 2.          9.832594    2.0888138   2.9154384   5.5903015
    1.7534697 ]
  ...
  [ 0.          0.          0.          0.          0.
    0.        ]
  [ 0.          0.          0.          0.          0.
    0.        ]
  [ 0.          0.          0.          0.          0.
    0.        ]]

 [[ 3.          9.186571    2.0888138   2.9154384   5.5903015
    1.7534697 ]
  [ 3.          8.401